# Advent of code 2020: day 14

Problem [here](https://adventofcode.com/2020/day/14)

In [1]:
print(bin(0xfa), 0xfa)
print(int("11111010", 2))
print(bin((0x1<<36)-1))

0b11111010 250
250
0b111111111111111111111111111111111111


## Part 1

In [2]:
class BitMask:
    def __init__(self, strRepr):
        self.mask = int("".join(("0" if ch == "X" else "1") for ch in strRepr), 2)
        self.val  = int("".join(("0" if ch == "X" else ch) for ch in strRepr), 2)
    def mod1(self, num):
        return (num&(~self.mask)) + (self.val&self.mask)
    def __str__(self):
        msk_s = f"{self.mask:036b}"
        val_s = f"{self.val:036b}"
        return "".join(("X" if mc == "0" else vc) for mc,vc in zip(msk_s, val_s))

ex_mask = BitMask("XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X")
print(str(ex_mask))

XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X


In [3]:
for ex_val in (11, 101, 0):
    print(f"value:  {ex_val:036b} (decimal {ex_val})")
    print(f"mask:   {ex_mask!s}")
    res = ex_mask.mod1(ex_val)
    print(f"result: {res:036b} (decimal {res})")

value:  000000000000000000000000000000001011 (decimal 11)
mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
result: 000000000000000000000000000001001001 (decimal 73)
value:  000000000000000000000000000001100101 (decimal 101)
mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
result: 000000000000000000000000000001100101 (decimal 101)
value:  000000000000000000000000000000000000 (decimal 0)
mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
result: 000000000000000000000000000001000000 (decimal 64)


In [4]:
def init(mem, lines, msk=None):
    if not msk:
        msk = BitMask("X"*36)
    for ln in lines:
        cmd, val = ln.split(" = ")
        if cmd == "mask":
            msk = BitMask(val)
        elif cmd.startswith("mem[") and cmd.endswith("]"):
            ky = int(cmd[4:-1])
            mem[ky] = msk.mod1(int(val))
        else:
            raise RuntimeError()
    return msk

In [5]:
ex_in = list("""mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0""".split("\n"))

from collections import defaultdict
ex_mem = defaultdict(int)
msk = init(ex_mem, ex_in)
print(sum(ex_mem.values()), str(msk), ex_mem)

165 XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X defaultdict(<class 'int'>, {8: 64, 7: 101})


In [6]:
with open("inputs/day14.txt") as inF:
    puz_lines = [ ln.strip() for ln in inF if ln.strip() ]

mem = defaultdict(int)
msk = init(mem, puz_lines)
print(msk, sum(mem.values()))

010X01011000XXXX1001101X00X01001XX01 13727901897109


## Part 2

In [7]:
ex_mask = BitMask("000000000000000000000000000000X1001X")
adr = 0b000000000000000000000000000000101010
print(f"address:  {adr:036b} (decimal {adr})")
print(f"mask:     {ex_mask!s}")
print(f"result:   {adr|ex_mask.val:036b}")
print(f"result-m: {(adr|ex_mask.val)&ex_mask.mask:036b}")

address:  000000000000000000000000000000101010 (decimal 42)
mask:     000000000000000000000000000000X1001X
result:   000000000000000000000000000000111010
result-m: 000000000000000000000000000000011010


In [8]:
def adrFor(adr, aMask):
    toCh = []
    for i in range(36):
        test = 0x1<<i
        if (~aMask) & test: ## is an X
            toCh.append(test)
    vals = []
    for i in range(2**len(toCh)):
        yield (adr&aMask) + sum((ch if i&(0x1<<j) else 0) for j,ch in enumerate(toCh))

ex_adr = 0b000000000000000000000000000000101010
ex_msk = BitMask("000000000000000000000000000000X1001X")
print("\n".join(f"{v:036b}" for v in adrFor(ex_adr|ex_mask.val, ex_msk.mask)))

000000000000000000000000000000011010
000000000000000000000000000000011011
000000000000000000000000000000111010
000000000000000000000000000000111011


In [9]:
ex_adr2 = 0b000000000000000000000000000000011010
ex_msk2 = BitMask("00000000000000000000000000000000X0XX")
print("\n".join(f"{v:036b}" for v in adrFor(ex_adr2|ex_msk2.val, ex_msk2.mask)))

000000000000000000000000000000010000
000000000000000000000000000000010001
000000000000000000000000000000010010
000000000000000000000000000000010011
000000000000000000000000000000011000
000000000000000000000000000000011001
000000000000000000000000000000011010
000000000000000000000000000000011011


In [10]:
def sumMemory(inLines):
    initInstr = []
    i = 0
    mem = dict()
    msk = None
    while i != len(inLines):
        cmd, val = inLines[i].split(" = ")
        if cmd == "mask":
            msk = BitMask(inLines[i].split(" = ")[1])
        elif cmd.startswith("mem[") and cmd.endswith("]"):
            ky = int(cmd[4:-1])
            val = int(val)
            for adr in adrFor(ky|msk.val, msk.mask):
                mem[adr] = val
        else:
            raise RuntimeError("Unknown command")
        i += 1
    return sum(mem.values())

ex2_in = list("""mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1""".split("\n"))


print(sumMemory(ex2_in))

208


In [11]:
print(sumMemory(puz_lines))

5579916171823
